In [10]:
import pandas as pd
import numpy as np 
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [3]:
# GDP_NOM_SECTORS: Nominal GDP of different sectors
# GDP_YOY_SECTORS: Year-on-year real GDP growth rates for different sectors
# AR: Agriculture, Forestry and Fisheries
# ID: Industrial
# CO: Construction
# WH: Wholesale and retail trade
# TR: Transportation, storage and postal services
# AC: Accommodation and catering
# FI: Finance
# RE: Real Estate
# IN: Information transmission, software and information technology services
# LE: Leasing and business services
# OT: Other Industries

In [11]:
GDP_NOM_SECTORS = pd.read_csv('GDP_NOM_SECTORS.csv').set_index('QTR')
GDP_YOY_SECTORS = pd.read_csv('GDP_YOY_SECTORS.csv').set_index('QTR')
GDP_YOY = pd.read_csv('GDP_YOY.csv').set_index('QTR')
GDP_YOY_SECTORS['OT'] = (GDP_NOM_SECTORS[['IN', 'LE', 'OT']].sum(axis = 1) / np.multiply(GDP_NOM_SECTORS[['IN', 'LE', 'OT']] , 1 / (1 + GDP_YOY_SECTORS[['IN', 'LE', 'OT']]/ 100)).sum(axis = 1) - 1 ) * 100
GDP_YOY_SECTORS = GDP_YOY_SECTORS.drop(columns=['IN','LE'])
GDP_NOM_SECTORS['OT'] = GDP_NOM_SECTORS[['IN', 'LE', 'OT']].sum(axis = 1)
GDP_NOM_SECTORS = GDP_NOM_SECTORS.drop(columns=['IN','LE'])
GDP_YOY_SECTORS

,AR,ID,CO,WH,TR,AC,FI,RE,OT
QTR,,,,,,,,,
1992-03,8.9,18.3,18.1,11.7,7.8,24.2,4.0,18.7,8.000000
1992-06,6.1,20.0,20.6,10.2,9.4,25.5,5.9,21.6,12.700000
1992-09,3.4,21.0,22.1,11.8,16.0,29.6,8.2,27.9,14.400000
1992-12,3.5,24.8,21.6,8.8,8.3,28.5,7.8,34.7,11.800000
1993-03,5.0,20.8,22.9,9.5,10.7,11.0,10.1,17.4,13.600000
...,...,...,...,...,...,...,...,...,...
2021-12,6.4,3.8,-2.1,5.9,4.0,4.7,5.5,-2.9,6.085922
2022-03,6.1,6.4,1.4,3.9,2.1,-0.3,5.1,-2.0,5.928831
2022-06,4.5,0.4,3.6,-1.8,-3.5,-5.3,5.9,-7.0,0.814498


In [13]:
from rpy2.robjects import r
from rpy2.robjects.packages import importr
from rpy2.robjects import globalenv
from rpy2.robjects import pandas2ri
pandas2ri.activate()

BVAR = importr("BVAR")

In [41]:
_lambda = 0.6
_alpha = 0.4
_lag = 6
gdp_yoy_sectors_pred_lag0 = []
gdp_yoy_sectors_pred_lag1 = []

rdf = pandas2ri.py2rpy(GDP_YOY_SECTORS.iloc[:80])
globalenv['rdf'] = rdf
globalenv['lambda'] = _lambda
globalenv['alpha'] = _alpha
globalenv['lag'] = _lag
rscript = """
bv_priors <- bv_mn( 
lambda = bv_lambda(mode = lambda ),
alpha = bv_alpha(mode = alpha), 
var = 10000000, 
b = 1
) 
x <- bvar(rdf, lags = lag, n_draw = 1000L, n_burn = 200L, priors = bv_priors(), verbose = FALSE)
predict(x) <- predict(x, horizon = 2)
irf(x) <- irf(x, horizon = 2, fevd = FALSE)
summary(x)
"""
result = r(rscript)
gdp_yoy_sectors_pred_lag0.append(np.median(result[0][10][0],  axis =0)[0])
gdp_yoy_sectors_pred_lag1.append(np.median(result[0][10][0],  axis =0)[1])

In [42]:
np.sum(np.array(GDP_NOM_SECTORS.iloc[80] / GDP_NOM_SECTORS.iloc[80].sum()) * gdp_yoy_sectors_pred_lag0, axis = 1)

array([8.81516627])

In [44]:
GDP_YOY_SECTORS.iloc[80]

AR     3.8
ID     9.4
CO    10.2
WH    10.1
TR     5.7
AC     6.2
FI     6.8
RE     0.1
OT     9.0
Name: 2012-03, dtype: float64